# Isoscape dataset extraction

Author: Martina Kauzlaric (martina.kauzlaric@unibe.ch)

This notebook is used to extract data from monthly isoscapes (i.e. isotopic landscapes, which are spatially continuous and georeferenced isotope datasets) of deuterium [‰] and oxygen-18 [‰] with a resolution of 500m provided by the FOEN into a table for publication alongisde the used data.
Stable isotope data in precipitation of the isotope observation network in Switzerland (ISOT), together with several influencing variables (e.g., topographical parameters, climate variables) are used in a multi-regression framework, and the residuals are interpolated by the use of ordinary kriging.
Monthly data are available for the years between 2007 and 2023.

## Requirements
**Python:**

* Python=3.13.2
* Jupyter
* os
* numpy=2.2.4
* xarray=2024.11.0
* pandas=2.2.3
* geopandas=1.0.1
* cartopy=0.24.1
* matplotlib=3.10.0
* tqdm=4.67.1

Check the Github repository for an environment_landcover.yml (for conda environments) file [for semplicity we use the same environment and that used for extracting the landcover data]

**Files:**

* ?


**Directory:**

* Clone the GitHub directory locally
* Place any third-data variables in their respective directory.
* ONLY update the "PATH" variable in the section "Configurations", with their relative path to the EStreams directory. 

## References
* Bundesamt für Umwelt BAFU
* Office fédéral de l'environnement OFEV
* Ufficio federale dell'ambiente UFAM
* Federal Office for the Environment FOEN
* © BAFU
* https://www.bafu.admin.ch/bafu/en/home/topics/water/groundwater/groundwater-resources/stable-water-isotopes.html
* https://www.bafu.admin.ch/dam/bafu/de/dokumente/hydrologie/externe-studien-berichte/isoscapes_schweiz_endbericht.pdf.download.pdf/isoscapes_schweiz_endbericht.pdf
## Observations
* Data are only available for catchments inside the national boundaries!

# Import modules

In [ ]:
# Clear all variables
%reset -f
#Import necessary libraries
import os
import glob
import numpy as np
#import xarray as xr
import geopandas as gpd
import pandas as pd
from shapely.geometry import MultiPolygon
from shapely.geometry import box
import tqdm as tqdm
import re
import rasterio
from rasterio.mask import mask
from concurrent.futures import ThreadPoolExecutor # this is to run functions in parallel

# Configurations

In [ ]:
# Only editable variables:
# Set (relative) path to your local directory
#PATH = "../.."
PATH = "S:\\CAMELS-CH\\CAMELS-chem"

In [ ]:
## Set directories
GIS_dir = os.path.join(PATH,"data\\GIS")
# Define shapefile with the catchments
catchments_shp = os.path.join(GIS_dir,"shapefile_catchments\\camels_ch_chem_catchment_boundaries.shp")
#Add subfolder to GIS_dir for Isoscape data
GIS_dir = os.path.join(GIS_dir, "Isoscapes")  
PATH_OUTPUT = os.path.join(PATH,"results\\catchment_aggregated_data\\isoscapes")

# Create the output directory if it does not exist
if not os.path.isdir(PATH_OUTPUT):
    os.makedirs(PATH_OUTPUT, exist_ok=True)

##Change to directory to where you want to store the results    
os.chdir(PATH_OUTPUT)

In [ ]:
os.getcwd()

* #### The users should NOT change anything in the code below here. 

# Import data
* Load catchments and look at full table

*Note: data are in LV95/CH1903+, i.e. EPSG 2056*

In [ ]:
catchments = gpd.read_file(catchments_shp)
catchments["bafu_id"] = catchments["gauge_id"]
catchments

Now we get the isoscapes and extract these per catchment.

*Note: data are in LV95/CH1903+, i.e. EPSG 2056*

In [ ]:
# Detect directories containing monthly isoscape data
ascii_dirs = [d for d in os.listdir(GIS_dir) if "Monatsraster" in d]
print(ascii_dirs)

In [ ]:
# Define isotopes to be extracted
isotopes = ["18O", "2H"]

In [ ]:
# --- HELPER FUNCTION FOR ONE CATCHMENT ---
def process_catchment(args):
    catch_idx, catch, raster_files, isotope, crs, isotope_outdir = args

    catch_id = catch["gauge_id"]
    catch_geom = gpd.GeoDataFrame([catch], crs=crs)

    values = []
    months = []

    for file_path in raster_files:
        month_match = re.search(r"_(\d{8})\.asc$", file_path)
        if not month_match:
            continue
        date_str = month_match.group(1)
        date = pd.to_datetime(date_str, format="%Y%m%d")
        months.append(date)

        try:
            with rasterio.open(file_path) as src:
                out_image, _ = mask(src, catch_geom.geometry, crop=True)
                out_image = out_image[0]
                valid = out_image != src.nodata
                if np.any(valid):
                    mean_value = float(np.nanmean(out_image[valid]))
                else:
                    mean_value = np.nan
        except Exception as e:
            print(f"⚠️ Error in file {file_path} for {catch_id}: {e}")
            mean_value = np.nan

        values.append(mean_value)

    # Save timeseries
    df = pd.DataFrame({
        "date": months,
        f"{isotope}_monthlymean": values
    })
    df.to_csv(os.path.join(isotope_outdir, f"{catch_id}_isoscape.csv"),
              index=False, sep=";", float_format="%.2f")
    return catch_id

In [ ]:
# --- MAIN LOOP ---
if __name__ == "__main__":
    for isotope in isotopes:
        print(f"\n🧪 Processing isotope {isotope}...")

        dir_match = [d for d in ascii_dirs if isotope in d]
        if not dir_match:
            print(f"⚠️ No directory found for isotope {isotope}")
            continue

        ascii_dir = os.path.join(GIS_dir, dir_match[0])
        raster_files = sorted([
            os.path.join(ascii_dir, f) for f in os.listdir(ascii_dir) if f.endswith(".asc")
        ])

        isotope_outdir = os.path.join(PATH_OUTPUT, isotope)
        os.makedirs(isotope_outdir, exist_ok=True)

        task_args = [
            (idx, row, raster_files, isotope, catchments.crs, isotope_outdir)
            for idx, row in catchments.iterrows()
        ]

        print(f"🚀 Launching parallel processing on {min(20, os.cpu_count())} cores...")

        with ThreadPoolExecutor(max_workers=20) as executor:
            results = list(tqdm.tqdm(executor.map(process_catchment, task_args),
                                     total=len(task_args), desc=f"Catchments for {isotope}"))

        print(f"Done with isotope {isotope} — {len(results)} catchments processed.")

Here  following is the code to loop over one catchment per time [depending on the number of catchments and the number of cores available maybe necessary..]

In [ ]:
# --- Helper Function ---
def extract_mean_per_catchment(raster_fp, catchment_geom):
    """Returns the mean of raster values within a catchment geometry."""
    with rasterio.open(raster_fp) as src:
        out_image, out_transform = mask(src, catchment_geom.geometry, crop=True)
        out_image = out_image[0]  # First band

        # Masked values are set to src.nodata
        valid = out_image != src.nodata
        if np.any(valid):
            return float(np.nanmean(out_image[valid]))
        else:
            return np.nan

In [ ]:
for isotope in isotopes:
    print(f"\n🧪 Processing isotope {isotope}...")

    # Find the correct directory for this isotope
    dir_match = [d for d in ascii_dirs if isotope in d]
    if not dir_match:
        print(f"⚠️ No directory found for isotope {isotope}")
        continue

    ascii_dir = os.path.join(GIS_dir, dir_match[0])
    raster_files = sorted([os.path.join(ascii_dir, f) for f in os.listdir(ascii_dir) if f.endswith(".asc")])

    # Create output folder for this isotope
    isotope_outdir = os.path.join(PATH_OUTPUT, isotope)
    os.makedirs(isotope_outdir, exist_ok=True)

    # Loop over catchments
    for _, catch in tqdm.tqdm(catchments.iterrows(), total=len(catchments), desc=f"Catchments for {isotope}"):
        catch_id = catch["gauge_id"]
        catch_geom = gpd.GeoDataFrame([catch], crs=catchments.crs)

        values = []
        months = []

        for file_path in raster_files:
            # Extract date from filename
            month_match = re.search(r"_(\d{8})\.asc$", file_path)
            if not month_match:
                continue
            date_str = month_match.group(1)
            date = pd.to_datetime(date_str, format="%Y%m%d")
            months.append(date)

            mean_value = extract_mean_per_catchment(file_path, catch_geom)
            values.append(mean_value)

        # Save timeseries for this catchment
        df = pd.DataFrame({
            "date": months,
            f"{isotope}_monthlymean": values
        })
        df.to_csv(os.path.join(isotope_outdir, f"{catch_id}_isoscape.csv"),
                  index=False, sep=";", float_format="%.2f")

Eventually, we generate the yearly mean isoscapes:

In [ ]:
for isotope in isotopes:
    print(f"\n📈 Generating yearly summary for {isotope}...")

    # Directory with catchment time series for this isotope
    iso_dir = os.path.join(PATH_OUTPUT, isotope)
    files = [f for f in os.listdir(iso_dir) if f.endswith("_isoscape.csv")]

    # Preallocate result DataFrame
    yearly_data = {}

    for file in files:
        catch_id = file.replace("_isoscape.csv", "")
        filepath = os.path.join(iso_dir, file)

        df = pd.read_csv(filepath, sep=";", parse_dates=["date"])
        df["year"] = df["date"].dt.year

        # Column is like "18O_monthlymean" or "2H_monthlymean"
        val_col = [col for col in df.columns if col.endswith("_monthlymean")][0]

        yearly_mean = df.groupby("year")[val_col].mean()
        yearly_data[catch_id] = yearly_mean

    # Combine into one DataFrame (rows = years, columns = catchments)
    yearly_df = pd.DataFrame(yearly_data)
    yearly_df.index.name = "year"

    # Save to CSV
    out_fp = os.path.join(PATH_OUTPUT, f"{isotope}_isoscape_yearly_summary.csv")
    yearly_df.to_csv(out_fp, sep=";", float_format="%.2f")

    print(f"Saved yearly summary to: {out_fp}")

Organize the data as time-series for the dataset (stored at the results folder)

In [ ]:
import os
import pandas as pd

folder_2h = '../../results/isoscapes/2H'
folder_18o = '../../results/isoscapes/18O'
output_folder = "../../results/Dataset/catchment_aggregated_data/rain_water_isotopes"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# List all files in the 2H folder
files_2h = sorted([f for f in os.listdir(folder_2h) if f.endswith('.csv')])

for filename in files_2h:
    path_2h = os.path.join(folder_2h, filename)
    path_18o = os.path.join(folder_18o, filename)

    # Read both files
    df_2h = pd.read_csv(path_2h, sep=';', parse_dates=['date'])
    df_18o = pd.read_csv(path_18o, sep=';', parse_dates=['date'])

    # Rename columns
    df_2h.rename(columns={'2H_monthlymean': 'delta_2h'}, inplace=True)
    df_18o.rename(columns={'18O_monthlymean': 'delta_18o'}, inplace=True)

    # Merge on date
    df_merged = pd.merge(df_2h, df_18o, on='date')

    # Set index
    df_merged.set_index('date', inplace=True)
    df_merged = df_merged.loc[:"2020", :]
    # Save the output
    output_path = os.path.join(output_folder, f"camels_ch_chem_rainisotopes_{filename[0:4]}.csv")
    df_merged.to_csv(output_path)